In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['Y_train.txt', 'X_test.txt', 'X_train.txt']


In [3]:
X = np.genfromtxt('../input/X_train.txt', delimiter = None)
Y = np.genfromtxt('../input/Y_train.txt', delimiter = None)
np.random.shuffle(X)
np.random.shuffle(Y)

In [10]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K

In [5]:
X_tr = X[10000:]
Y_tr = Y[10000:]
X_va = X[:10000]
Y_va = Y[:10000]

In [6]:
X_tr.shape

(90000, 14)

In [11]:
df = pd.DataFrame()
for i in range(14):
    df["Feature%s" % i] = preprocessing.scale(X_tr[:, i])

In [12]:
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# AUC for a binary classifier
def auc(y_true, y_pred):   
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)
    binSizes = -(pfas[1:]-pfas[:-1])
    s = ptas*binSizes
    return K.sum(s, axis=0)

#-----------------------------------------------------------------------------------------------------------------------------------------------------
# PFA, prob false alert for binary classifier
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # N = total number of negative labels
    N = K.sum(1 - y_true)
    # FP = total number of false alerts, alerts from the negative class labels
    FP = K.sum(y_pred - y_pred * y_true)    
    return FP/N
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# P_TA prob true alerts for binary classifier
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # P = total number of positive labels
    P = K.sum(y_true)
    # TP = total number of correct alerts, alerts from the positive class labels
    TP = K.sum(y_pred * y_true)    
    return TP/P

In [13]:
model = Sequential()
model.add(Dense(20, kernel_initializer='normal', activation='relu'))
model.add(Dense(5, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.compile(loss = 'mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [14]:
model.fit(df.values, Y_tr, epochs = 2, batch_size = 20)

Epoch 1/2
90000/90000 [==============================] - 6s 62us/step - loss: 0.2270 - acc: 0.6579
Epoch 2/2
90000/90000 [==============================] - 5s 56us/step - loss: 0.2252 - acc: 0.6583


In [18]:
from sklearn import svm, metrics

In [16]:
clf = svm.SVC(kernel='linear')
clf.fit(df.values, Y_tr)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [17]:
y_pred = clf.predict(X_va)

In [19]:
metrics.accuracy_score(Y_va, y_pred)

0.663